# Accessing existing tables in Spark

In this notebook we will access the Boston dataset stored in Spark warehouse in previous notebook.

In [1]:
from pyspark.sql import SparkSession;

# warehouse_location points to the default location for managed databases and tables
from os.path import abspath
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    . builder \
    .master("local[*]") \
    .appName("ISM6562 PySpark Tutorials") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

sc =spark.sparkContext
sc.setLogLevel("ERROR") # only display errors (not warnings)

# note: If you have multiple spark sessions running (like from a previous notebook you've run), 
# this spark session webUI will be on a different port than the default (4040). One way to 
# identify this part is with the following line. If there was only one spark session running, 
# this will be 4040. If it's higher, it means there are still other spark sesssions still running.
spark_session_port = spark.sparkContext.uiWebUrl.split(":")[-1]
print("Spark Session WebUI Port: " + spark_session_port)

23/10/27 17:24:07 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 10.21.5.100 instead (on interface eth0)
23/10/27 17:24:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/27 17:24:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session WebUI Port: 4040


Display the spark object - this provides the link to the Spark UI

In [2]:
spark

List the Spark datawarehouse. It should show the default database.

In [3]:
spark.catalog.listTables()

23/10/27 17:24:11 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/10/27 17:24:11 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/10/27 17:24:14 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/10/27 17:24:14 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore student@127.0.0.1
23/10/27 17:24:15 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


[Table(name='fake_friends', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='incidents', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='movieratings', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='movies', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False)]

In [4]:
df=spark.sql("show databases;") 
df.show()

+---------+
|namespace|
+---------+
|  default|
|   w10_db|
+---------+



List the tables in the database. If this is a new install (you haven't run this notebook before), there shouldn't be any tables.

In [5]:
tables = spark.sql("show tables").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|  default|fake_friends|      false|
|  default|   incidents|      false|
|  default|movieratings|      false|
|  default|      movies|      false|
+---------+------------+-----------+



We can use SQL to query the table. The result is a dataframe.

In [6]:
df = spark.sql("select * from w10_db.boston")
df.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+-----+----+---------+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM| AGE|   DIS|RAD|TAX|PTRATIO|LSTAT|MEDV|CAT. MEDV|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+-----+----+---------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 4.98|24.0|        0|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 9.14|21.6|        0|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8| 4.03|34.7|        1|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7| 2.94|33.4|        1|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 5.33|36.2|        1|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+-----+----+---------+
only showing top 5 rows



But notice that the temp view is not available...

In [7]:
#boston.createOrReplaceTempView("boston_tmp_view")

In [8]:
spark.catalog.listTables()

[Table(name='fake_friends', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='incidents', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='movieratings', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='movies', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False)]

In [9]:
spark.catalog.listTables('w10_db')

[Table(name='boston', catalog='spark_catalog', namespace=['w10_db'], description=None, tableType='MANAGED', isTemporary=False)]

In [10]:
spark.stop()